# Zadanie domowe -- interpolacja dwusześcienna

Interpolacja dwusześcienna, to podobnie jak w przypadku interpolacji dwuliniowej, rozszerzenie idei interpolacji jednowymiarowej na dwuwymiarową siatkę.
W trakcie jej obliczania wykorzystywane jest 16 pikseli z otoczenia (dla dwuliniowej 4).
Skutkuje to zwykle lepszymi wynikami - obraz wyjściowy jest bardziej gładki i z mniejszą liczbą artefaktów.
Ceną jest znaczny wzrost złożoności obliczeniowej (zostało to zaobserwowane podczas ćwiczenia).

Interpolacja dana jest wzorem:
\begin{equation}
I(i,j) = \sum_{i=0}^{3} \sum_{j=0}^{3} a_{ij} x^i y^j
\end{equation}

Zadanie sprowadza się zatem do wyznaczenia 16 współczynników $a_{ij}$.
W tym celu wykorzystuje się, oprócz wartość w~puntach $A$ (0,0), $B$ (1 0), $C$ (1,1), $D$ (0,1) (por. rysunek dotyczący interpolacji dwuliniowej), także pochodne cząstkowe $A_x$, $A_y$, $A_{xy}$.
Pozwala to rozwiązać układ 16-tu równań.

Jeśli zgrupujemy parametry $a_{ij}$:
\begin{equation}
a = [ a_{00}~a_{10}~a_{20}~a_{30}~a_{01}~a_{11}~a_{21}~a_{31}~a_{02}~a_{12}~a_{22}~a_{32}~a_{03}~a_{13}~a_{23}~a_{33}]
\end{equation}

i przyjmiemy:
\begin{equation}
x = [A~B~D~C~A_x~B_x~D_x~C_x~A_y~B_y~D_y~C_y~A_{xy}~B_{xy}~D_{xy}~C_{xy}]^T
\end{equation}

To zagadnienie można opisać w postaci równania liniowego:
\begin{equation}
Aa = x
\end{equation}
gdzie macierz $A^{-1}$ dana jest wzorem:

\begin{equation}
A^{-1} =
\begin{bmatrix}
1& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0 \\
0&  0&  0&  0&  1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
-3&  3&  0&  0& -2& -1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
2& -2&  0&  0&  1&  1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  1&  0&  0&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  1&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0& -3&  3&  0&  0& -2& -1&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  2& -2&  0&  0&  1&  1&  0&  0 \\
-3&  0&  3&  0&  0&  0&  0&  0& -2&  0& -1&  0&  0&  0&  0&  0 \\
0&  0&  0&  0& -3&  0&  3&  0&  0&  0&  0&  0& -2&  0& -1&  0 \\
9& -9& -9&  9&  6&  3& -6& -3&  6& -6&  3& -3&  4&  2&  2&  1 \\
-6&  6&  6& -6& -3& -3&  3&  3& -4&  4& -2&  2& -2& -2& -1& -1 \\
2&  0& -2&  0&  0&  0&  0&  0&  1&  0&  1&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  2&  0& -2&  0&  0&  0&  0&  0&  1&  0&  1&  0 \\
-6&  6&  6& -6& -4& -2&  4&  2& -3&  3& -3&  3& -2& -1& -2& -1 \\
4& -4& -4&  4&  2&  2& -2& -2&  2& -2&  2& -2&  1&  1&  1&  1 \\
\end{bmatrix}
\end{equation}

Potrzebne w rozważaniach pochodne cząstkowe obliczane są wg. następującego przybliżenia (przykład dla punktu A):
\begin{equation}
A_x = \frac{I(i+1,j) - I(i-1,j)}{2}
\end{equation}

\begin{equation}
A_y = \frac{I(i,j+1) - I(i,j-1)}{2}
\end{equation}

\begin{equation}
A_xy = \frac{I(i+1,j+1) - I(i-1,j) - I(i,j-1) + I(i,j)}{4}
\end{equation}

## Zadanie

Wykorzystując podane informacje zaimplementuj interpolację dwusześcienną.
Uwagi:
- macierz $A^{-1}$ dostępna jest w pliku *a_invert.py*
- trzeba się zastanowić nad potencjalnym wykraczaniem poza zakres obrazka (jak zwykle).

Ponadto dokonaj porównania liczby operacji arytmetycznych i dostępów do pamięci koniecznych przy realizacji obu metod interpolacji: dwuliniowej i dwusześciennej.

In [18]:
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt

if not os.path.exists("ainvert.py") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/05_Resolution/ainvert.py --no-check-certificate

import ainvert

In [19]:
def bicubic(image, ver, hor):
    x = image.shape[0]
    y = image.shape[1]
    x_new = int(np.round(ver * x))
    y_new = int(np.round(hor * y))

    new_image = np.zeros((x_new, y_new))

    for i in range(1, x_new - 3):
        for j in range(1, y_new - 3):
            i_new = i / ver
            j_new = j / hor

            i1 = int(np.floor(i_new))
            j1 = int(np.floor(j_new))
            i2 = int(np.floor(i_new) + 1)
            j2 = int(np.floor(j_new) + 1)

            # if np.floor(i_new) + 1 >= x:
            #     i2 = x - 1
            # if np.floor(j_new) + 1 >= y:
            #     j2 = y - 1

            if i1 == i2:
                i1 -= 1
            if j1 == j2:
                j1 -= 1
            if i_new > x-1:
                i_new -= 1
            if j_new > x-1:
                j_new -= 1

            A = image[i1, j1]
            B = image[i1, j2]
            C = image[i2, j1]
            D = image[i2, j2]

            Ax = (image[i1 + 1, j1] - image[i1 - 1, j1]) / 2
            Ay = (image[i1, j1 + 1] - image[i1, j1 - 1]) / 2
            Axy = (image[i1 + 1, j1 + 1] - image[i1 - 1, j1] -
                   image[i1, j1 - 1] + image[i1, j1]) / 4

            Bx = (image[i1 + 1, j2] - image[i1 - 1, j2]) / 2
            By = (image[i1, j2 + 1] - image[i1, j2 - 1]) / 2
            Bxy = (image[i1 + 1, j2 + 1] - image[i1 - 1, j2] -
                   image[i1, j2 - 1] + image[i1, j2]) / 4

            Dx = (image[i2 + 1, j1] - image[i2 - 1, j1]) / 2
            Dy = (image[i2, j1 + 1] - image[i2, j1 - 1]) / 2
            Dxy = (image[i2 + 1, j1 + 1] - image[i2 - 1, j1] -
                   image[i2, j1 - 1] + image[i2, j1]) / 4

            Cx = (image[i2 + 1, j2] - image[i2 - 1, j2]) / 2
            Cy = (image[i2, j2 + 1] - image[i2, j2 - 1]) / 2
            Cxy = (image[i2 + 1, j2 + 1] - image[i2 - 1, j2] -
                   image[i2, j2 - 1] + image[i2, j2]) / 4

            x = [A, B, D, C, Ax, Bx, Dx, Cx, Ay,
                 By, Dy, Cy, Axy, Bxy, Dxy, Cxy]

            x = np.reshape(x, (len(x), 1))

            a = np.dot(ainvert.A_invert, x)

            a = np.reshape(a, (4, 4))

            for k in range(4):
                for l in range(4):
                    new_image[i, j] += a[k, l] * (i ** k) * (j ** l)
    return new_image

def img_show(image):
    plt.imshow(image, cmap="gray")
    plt.xticks([]), plt.yticks([])  # Hides the graph ticks and x / y axis
    plt.show()

In [20]:
if not os.path.exists("parrot.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/05_Resolution/parrot.bmp --no-check-certificate

parrot = cv2.imread('parrot.bmp')           # Read image
parrot = cv2.cvtColor(parrot, cv2.COLOR_BGR2GRAY) # Convert to RGB

In [21]:
new_parrot = bicubic(parrot, 2, 2)
img_show(new_parrot)

/var/folders/_1/w52qf3p97d761kcqp1lbqq480000gn/T/ipykernel_90688/3686493070.py:38: RuntimeWarning: overflow encountered in scalar subtract
  Ax = (image[i1 + 1, j1] - image[i1 - 1, j1]) / 2
/var/folders/_1/w52qf3p97d761kcqp1lbqq480000gn/T/ipykernel_90688/3686493070.py:40: RuntimeWarning: overflow encountered in scalar subtract
  Axy = (image[i1 + 1, j1 + 1] - image[i1 - 1, j1] -
/var/folders/_1/w52qf3p97d761kcqp1lbqq480000gn/T/ipykernel_90688/3686493070.py:43: RuntimeWarning: overflow encountered in scalar subtract
  Bx = (image[i1 + 1, j2] - image[i1 - 1, j2]) / 2
/var/folders/_1/w52qf3p97d761kcqp1lbqq480000gn/T/ipykernel_90688/3686493070.py:45: RuntimeWarning: overflow encountered in scalar subtract
  Bxy = (image[i1 + 1, j2 + 1] - image[i1 - 1, j2] -
/var/folders/_1/w52qf3p97d761kcqp1lbqq480000gn/T/ipykernel_90688/3686493070.py:48: RuntimeWarning: overflow encountered in scalar subtract
  Dx = (image[i2 + 1, j1] - image[i2 - 1, j1]) / 2
/var/folders/_1/w52qf3p97d761kcqp1lbqq480000gn

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()